In [182]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta 

df = pd.DataFrame()
os.getcwd()
 

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [266]:
# pd.read_excel('./data/Coach11-1to11-5.xlsx', sheet_name=0)\
#     .to_csv('./data/coach_15.csv', encoding = 'utf_8_sig', index=None)
coach = pd.read_csv('./data/coach_15.csv')
coach = coach.dropna(axis=1, how = 'all')
# task_collect = coach.groupby(['姓名', '开始日期'])['任务类型'].apply(list).astype(str).reset_index();task_collect
# onoff = task_collect[task_collect['任务类型'].str.contains('(下班.+上班)')][['姓名','开始日期']].sum(axis = 1)
# coach['flag'] = coach[['姓名','开始日期']].sum(axis = 1)
coach['开始时间'] = pd.to_datetime(coach['开始时间'])
coach['结束时间'] = pd.to_datetime(coach['结束时间'])

# 标准时长计算
coach = pd.concat([coach, pd.DataFrame(list(coach['调整后持续时间'].str.split(':')))], axis =1).drop('序号',axis =1)
coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
coach[[0,1,2]] = coach[[0,1,2]].astype(int)
coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
coach = coach.drop([0,1,2], axis = 1)
# coach_nopunch = coach[~coach['flag'].isin(onoff)]
# coach_dopunch = coach[ coach['flag'].isin(onoff)]
coach['sprm'] = (60/coach['SPR标准数量']).replace([np.inf, -np.inf], 0)

In [268]:
wh = coach.groupby(['姓名','开始日期']).agg(
    {
        '开始时间': 'min',
        '结束时间': 'max',
        'sprm': 'sum',
    }
).reset_index()
wh['work_hour'] = wh['结束时间'] - wh['开始时间']
wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)

In [269]:
coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')

In [274]:
# coach[coach['姓名'] == '胡涛']

In [ ]:
coach.groupby(['姓名', ''])

In [283]:
# SELECT  * FROM dm_dsc_smart.dwd_task
# where substr(station_name, 1,4) = 'COAC'
# and work_group_id != 0 and work_group_name != ''
# and inc_day = '20211127'
 

In [371]:
coach2 = pd.read_csv('./data/coach_all_bdp.csv', sep = '\001')
coach2 = coach2.dropna(how = 'all', axis = 1)
coach2.columns = [re.sub('\w+\.', '', i) for i in list(coach2.columns)]

In [378]:
# # coach2= coach2.drop(['dwd_task_id', 'station_id', 'work_turn_id', 'raw_data'], axis  =1)

# coach2['work_content'].value_counts()
# coach2['work_content_refer'].value_counts()

In [364]:
# np.arange(coach2.shape[0])

In [361]:
df = pd.DataFrame()
for i in np.arange(coach2.shape[0]):
    df = pd.concat([pd.DataFrame.from_dict(json.loads(coach2['raw_data'][i]), orient="index" ).T, df], axis = 0)

In [372]:
# df['activity_type'].value_counts()

In [309]:
coach2[['station_name','worker_name', 'work_group_name', 'up_worker_name',\
    'hire_time', 'worker_post_name', 'worker_level_name', 'work_num', 'start_time', 'end_time', 'duration',
       'adjustment_duration', 'adjusted_duration', 'sku_no', 'min_package_quantity', 'min_operate_length', 'min_operate_width',
       'min_operate_height', 'min_operate_weight', 'min_operate_volume',
       'work_content', 'work_content_refer', 'work_content_is_measure',
       'start_date', 'end_date', 'report_date', 'create_time', 'update_time']]

,station_name,worker_name,work_group_name,up_worker_name,hire_time,worker_post_name,worker_level_name,work_num,start_time,end_time,...,min_operate_volume,work_content_id,work_content_type,work_content_is_measure,work_content_refer,start_date,end_date,report_date,create_time,update_time
0,COACH SHA WGQ WHS,彭庆伟,拣货组,孙波,1624809600,操作员,操作员,3.0,1637975498,1637975542,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
1,COACH SHA WGQ WHS,彭庆伟,拣货组,孙波,1624809600,操作员,操作员,0.0,1637975638,1637975645,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
2,COACH SHA WGQ WHS,彭庆伟,拣货组,孙波,1624809600,操作员,操作员,0.0,1637975798,1637975800,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
3,COACH SHA WGQ WHS,彭庆伟,拣货组,孙波,1624809600,操作员,操作员,1.0,1637976516,1637976531,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
4,COACH SHA WGQ WHS,彭庆伟,拣货组,孙波,1624809600,操作员,操作员,1.0,1637979834,1637979968,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,COACH SHA WGQ WHS,庞士赛,拣货组,李小丹,1630252800,操作员,操作员,2.0,1638002416,1638002483,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
116,COACH SHA WGQ WHS,庞士赛,拣货组,李小丹,1630252800,操作员,操作员,2.0,1638002501,1638002519,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
117,COACH SHA WGQ WHS,庞士赛,拣货组,李小丹,1630252800,操作员,操作员,7.0,1638002611,1638002687,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191
118,COACH SHA WGQ WHS,庞士赛,拣货组,李小丹,1630252800,操作员,操作员,2.0,1638003141,1638003194,...,0.0,0,0,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,1638231191


In [368]:
def time_convert(col):
    coach2[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach2[col]]
    return coach2

In [370]:
time_convert('update_time')

,dwd_task_id,account_id,account_name,station_id,station_code,station_name,worker_id,worker_name,work_turn_id,work_group_id,...,raw_data,is_cancel,efficiency,orig_task_type,orig_user_id,create_time,update_time,pallet_operation,inc_day,sys
0,wmos-wmos-264013182,11133758,彭庆伟,0,CN-085,COACH SHA WGQ WHS,26895,彭庆伟,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC005RF""...",0,0.0,500,ADC005RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
1,wmos-wmos-264014645,11133758,彭庆伟,0,CN-085,COACH SHA WGQ WHS,26895,彭庆伟,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC005RF""...",0,0.0,500,ADC005RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
2,wmos-wmos-264016651,11133758,彭庆伟,0,CN-085,COACH SHA WGQ WHS,26895,彭庆伟,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC005RF""...",0,0.0,500,ADC005RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
3,wmos-wmos-264023733,11133758,彭庆伟,0,CN-085,COACH SHA WGQ WHS,26895,彭庆伟,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC005RF""...",0,0.0,500,ADC005RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
4,wmos-wmos-264057913,11133758,彭庆伟,0,CN-085,COACH SHA WGQ WHS,26895,彭庆伟,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC005RF""...",0,0.0,500,ADC005RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,wmos-wmos-264254167,11240215,庞士赛,0,CN-085,COACH SHA WGQ WHS,27790,庞士赛,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC015RF""...",0,0.0,500,ADC015RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
116,wmos-wmos-264254495,11240215,庞士赛,0,CN-085,COACH SHA WGQ WHS,27790,庞士赛,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC015RF""...",0,0.0,500,ADC015RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
117,wmos-wmos-264255994,11240215,庞士赛,0,CN-085,COACH SHA WGQ WHS,27790,庞士赛,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC015RF""...",0,0.0,500,ADC015RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
118,wmos-wmos-264259256,11240215,庞士赛,0,CN-085,COACH SHA WGQ WHS,27790,庞士赛,0,10025701,...,"{""wms_warehouse_id"":""C25"",""user_id"":""ADC015RF""...",0,0.0,500,ADC015RF,2021-11-30 08:13:11,2021-11-30 08:13:11,0,20211127,wms
